# Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества **F1 не меньше 0.75**.

## Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять BERT необязательно, но вы можете попробовать.

## Описание данных

Данные находятся в файле `toxic_comments.csv`.

Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

Проверим наличие GPU и установим нужные зависимости.

In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
!pip install transformers

## Подготовка

In [3]:
import torch

import transformers as ppb
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification

from tqdm import notebook

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

import re

import pandas as pd
import numpy as np

import logging

import matplotlib.pyplot as plt
import seaborn as sns

import time
import datetime
import random

In [4]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
data.sample(10, random_state=123)

,text,toxic
50446,", and redirect the other names to it",0
81571,SineBot1\nPlease read the above comments.,0
25983,Thank you for your very good answer. I just r...,0
39022,"""\n I think we need to ask who is likely to be...",0
49431,"Orangemonster2k1|SVRTVDude]] (VT) 23:26, 30 April",0
79072,""" May 2009 (UTC)\n\n^^THANK GOD THERES SOMEBOD...",0
24907,Incan writing? \n\nDid the Incas have any writ...,0
108333,"""\nGive it up, you're """"pissing in the wind"""" ...",0
50023,"She did not die though, it's ok dont worry",0
157296,Fixed reference in the article.,0


In [7]:
data.toxic.value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

У нас датасет с 159571 неочищенными комментариями на английском языке и с большим дисбалансом классов. Пропусков в датасете нет.

Выделим признак и целевой признак, а так же разобьём на тренировачную и тестовую выборки.

In [8]:
text = data['text']
labels = data['toxic']

train_text, test_text, train_labels, test_labels = train_test_split(text, labels, test_size=.2, stratify=labels, random_state=123) 

Посмотрим на количество данных в каждой выборке.

In [9]:
print('Размер тренировочной выборки: {}'.format(len(train_text)))
print('Размер тестовой выборки: {}'.format(len(test_text)))
print()
print('Классы:', np.unique(labels))

Размер тренировочной выборки: 127656
Размер тестовой выборки: 31915

Классы: [0 1]


Ниже вы воспользуемся более простым способом для предсказывания токсичности комментария:
1. очистим тексты от лишних символов
2. лемматизируем наши тексты
3. уберем стоп слова
4. вычислим TF-IDF
5. создадим и натренируем модель логистической регрессии

In [10]:
def clean_text(value):
    value = re.sub(r"http\S+", "", value)
    value = re.sub(r"http", "", value)
    value = re.sub("([<>-])|[[:punct:]]", "\\1", value)
    value = re.sub("\n", " ", value)
    value = re.sub("(\\S+\\d+|\\d+)\\S+", "", value)
    value = re.sub("([<>-])|[[:punct:]]", "\\1", value)
    return value

In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
def preprocess_text(text):
    text = clean_text(text)

    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]

    return " ".join(lemmas)

In [13]:
train_corpus = train_text.apply(preprocess_text)

In [14]:
train_corpus.head()

135751    royal urban legends you want one link and he w...
116496                 and take almost everything literally
134498    schmuckythecat move the draft into articlespac...
39542     yup once the article be there add it to the li...
119967    also also your version be now slightly ambiguo...
Name: text, dtype: object

In [15]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

tf_idf_vectorizer = TfidfVectorizer(stop_words=stopwords)

train_tf_idf = tf_idf_vectorizer.fit_transform(train_corpus)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
train_tf_idf.shape

(127656, 134746)

In [17]:
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

grid = dict(solver=solvers,penalty=penalty,C=c_values)

logistic_regression_model = GridSearchCV(LogisticRegression(max_iter=10000), param_grid=grid,
                   cv=5, n_jobs = -2)

logistic_regression_model.fit(train_tf_idf, train_labels)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=10000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-2,
             param_grid={'C': [100, 10, 1.0, 0.1, 0.01], 'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [18]:
logistic_regression_model.best_params_

{'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}

In [19]:
test_corpus = test_text.apply(preprocess_text)
test_corpus.head()

24806     because you know everything hi sandstein if yo...
1956      consider the legal issue surround roms it woul...
143203    i be demand to speak to someone of the highest...
125561    re read the article i link to because that sam...
20953                             o dieeeeeeeeeeeeeeeeeeeee
Name: text, dtype: object

In [20]:
test_tf_idf = tf_idf_vectorizer.transform(test_corpus)

test_tf_idf.shape

(31915, 134746)

In [21]:
logistic_regression_predictions = logistic_regression_model.predict(test_tf_idf)

In [22]:
print('LogisticRegression F1-score: {}'.format(f1_score(test_labels, logistic_regression_predictions)))

LogisticRegression F1-score: 0.7674418604651163


Нам удалось достичь нужного результата на логистической регрессии f1-score 0.76, обучение и предсказания происходят быстро.

Попробуем более сложную модель, а именно Bert. Для эмбединга и классификации воспользуемся моделью DistilBertForSequenceClassification, загрузим 
претренированный токенайзер.

In [23]:
tokenizer_class, pretrained_weights = (ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [24]:
tokenizer = ppb.DistilBertTokenizer.from_pretrained(pretrained_weights, do_lower_case=True)

Возьмём сниппет функции помогающей отслеживать прогресс выполнения долгих операций, а их у нас будет много.

In [25]:
def good_update_interval(total_iters, num_desired_updates):
    exact_interval = total_iters / num_desired_updates
    order_of_mag = len(str(total_iters)) - 1
    round_mag = order_of_mag - 1
    update_interval = int(round(exact_interval, -round_mag))

    if update_interval == 0:
        update_interval = 1

    return update_interval

Напишем функцию создающая "умные" батчи:
1. токенизируем и обрежем наши комментарии по длине(максимальная длина для BERT 512 символов, мы возьмём 400)
2. Отсортируем наши последовательности батчей по из длине по возрастанию
3. Создадим батчи с размер в 16 сэмплов
4. Применим padding для каждого батча


In [26]:
max_len = 400
batch_size = 16

In [27]:
def make_smart_batches(text_samples, labels, batch_size):
    
    print('Создадим Smart Batches из {:,} текстов с размером батча {:,}...\n'.format(len(text_samples), batch_size))

    # ==============================================
    #   Токенизация и усечение последовательностей
    # ==============================================

    full_input_ids = []

    print('Токенизируем {:,} текстов...'.format(len(text_samples)))

    # выберем интервал для вывода прогресса выполнения.
    update_interval = good_update_interval(total_iters=len(text_samples), num_desired_updates=10)

    # токениризируем каждый текст в переданнос датасете
    for text in text_samples:
        
        # Вывод отчёта о прогрессе.
        if ((len(full_input_ids) % update_interval) == 0):
            print('  Токенизировано {:,} текстов.'.format(len(full_input_ids)))

        # токениризуем сэмпл
        input_ids = tokenizer.encode(text=text,              # текст.
                                    add_special_tokens=True, # добавим символы начала и конца строки
                                    max_length=max_len,      # ограничим наш текст 400 количеством символов
                                    truncation=True,         # включим ограничение
                                    padding=False)           # без применения паддинга
                                    
        full_input_ids.append(input_ids)
        
    print('DONE.')
    print('{:>10,} текстов\n'.format(len(full_input_ids)))

    # =========================
    #    определение батчей
    # =========================    

    # перед определением батчей, отсортируем на последовательности по длине по возрастанию
    samples = sorted(zip(full_input_ids, labels), key=lambda x: len(x[0]))

    print('{:>10,} текстов после сортировки\n'.format(len(samples)))

    

    # списки для хранения полученных батчей
    batch_ordered_sentences = []
    batch_ordered_labels = []

    print('Выборка батчей размера {:}...'.format(batch_size))

    # выберем интервал для вывода прогресса выполнения.
    update_interval = good_update_interval(total_iters=len(samples), num_desired_updates=10)
    
    while len(samples) > 0:
        
        # отчёт о прогрессе
        if ((len(batch_ordered_sentences) % update_interval) == 0 \
            and not len(batch_ordered_sentences) == 0):
            print('  Выборка батча {:,}.'.format(len(batch_ordered_sentences)))

        # сохраняем размер нашего батча, если это последний батч, то сохраняем остаток
        # который по размеру меньше батча
        to_take = min(batch_size, len(samples))

        # сгенерируем рандомный индекс с которого будет начинаться наш батч
        select = random.randint(0, len(samples) - to_take)

        # выберем наш батч
        batch = samples[select:(select + to_take)]

        # так как в сэмпле храниться и класс и токениризоравнный текст с помощью кортежа, 
        # то в один список мы записываем нашу последовательность, а во второй класс
        batch_ordered_sentences.append([s[0] for s in batch])
        batch_ordered_labels.append([s[1] for s in batch])

        # удаляем наш батч из выборки
        del samples[select:select + to_take]

    print('\n  DONE - {:,} батчей выбрано.\n'.format(len(batch_ordered_sentences)))

    # =========================
    #     Добавляем Padding
    # =========================    

    print('Padding последовательностей в каждом батче...')

    py_inputs = []
    py_attn_masks = []
    py_labels = []

    for (batch_inputs, batch_labels) in zip(batch_ordered_sentences, batch_ordered_labels):

        batch_padded_inputs = []
        batch_attn_masks = []
        
        # найдём размер саммой длинной последовательности в батче
        max_size = max([len(sen) for sen in batch_inputs])

        for sen in batch_inputs:
            
            # вычисляем количество токенов, которое на добавить
            num_pads = max_size - len(sen)

            # добавляем токены в последовательность
            padded_input = sen + [tokenizer.pad_token_id]*num_pads

            # добавим маску для выделения важных токенов
            attn_mask = [1] * len(sen) + [0] * num_pads

            # добавим результаты в списки
            batch_padded_inputs.append(padded_input)
            batch_attn_masks.append(attn_mask)

        # сохраним наши батчи и преобразуем их в тензоры PyTorch
        py_inputs.append(torch.tensor(batch_padded_inputs))
        py_attn_masks.append(torch.tensor(batch_attn_masks))
        py_labels.append(torch.tensor(batch_labels))
    
    print('  DONE.')

    # Возвращаем наши Smart batches
    return (py_inputs, py_attn_masks, py_labels)

In [28]:
(train_py_inputs, train_py_attn_masks, train_py_labels) = make_smart_batches(train_text, train_labels, batch_size)

Создадим Smart Batches из 127,656 текстов с размером батча 16...

Токенизируем 127,656 текстов...
  Токенизировано 0 текстов.
  Токенизировано 10,000 текстов.
  Токенизировано 20,000 текстов.
  Токенизировано 30,000 текстов.
  Токенизировано 40,000 текстов.
  Токенизировано 50,000 текстов.
  Токенизировано 60,000 текстов.
  Токенизировано 70,000 текстов.
  Токенизировано 80,000 текстов.
  Токенизировано 90,000 текстов.
  Токенизировано 100,000 текстов.
  Токенизировано 110,000 текстов.
  Токенизировано 120,000 текстов.
DONE.
   127,656 текстов

   127,656 текстов после сортировки

Выборка батчей размера 16...

  DONE - 7,979 батчей выбрано.

Padding последовательностей в каждом батче...
  DONE.


Получим конфиг для нашей претринерованной модели DistilBert.

In [29]:
config = AutoConfig.from_pretrained(pretrained_model_name_or_path='distilbert-base-uncased',
                                    num_labels=2)

print('Config type:', str(type(config)), '\n')

Config type: <class 'transformers.configuration_distilbert.DistilBertConfig'> 



Загрузим претринерованную модель DistilBert для классфикации, с пробросом конфига.

In [30]:
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path='distilbert-base-uncased',
    config=config)

print('\nModel type:', str(type(model)))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi


Model type: <class 'transformers.modeling_distilbert.DistilBertForSequenceClassification'>


Укажем модели использовать GPU.

In [31]:
print('\nLoading model to GPU...')

device = torch.device('cuda')

print('  GPU:', torch.cuda.get_device_name(0))

desc = model.to(device)

print('    DONE.')


Loading model to GPU...
  GPU: Tesla T4
    DONE.


Создадим оптимизаторы для нашей модели.

In [32]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = 5e-5, 
                  eps = 1e-8 
                )


In [33]:
from transformers import get_linear_schedule_with_warmup

epochs = 1
total_steps = len(train_py_inputs) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Добавим функцию форматирования даты для вывода оставшегося время работы.

In [34]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

Натренируем нашу модель

P.S. код тренировки модели взять из библиотеки transformers
https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


In [35]:
seed_val = 321

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Будем сохранять потери, accuracy и время выполнения
training_stats = []

# выберем интервал для вывода прогресса выполнения.
update_interval = good_update_interval(total_iters=len(train_py_inputs), num_desired_updates=10)

# замеряем общее время выполнения
total_t0 = time.time()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Тренировка
    # ========================================
    
    # Проходимся по всей тренировачной выборке за эпоху.

    print("")
    print('======== Эпоха {:} / {:} ========'.format(epoch_i + 1, epochs))
    
    # перед каждой эпохой снова рандомизируем нашу выборку
    if epoch_i > 0:
        (train_py_inputs, train_py_attn_masks, train_py_labels) = make_smart_batches(train_texts, train_labels, batch_size)
    
    print('Тренировка на {:,} батчей...'.format(len(train_py_inputs)))

    # замеряем как долго тренировалась одна эпоха
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # установим модель в режим тренировки
    model.train()
    for step in range(0, len(train_py_inputs)):

        # Вывод прогресса
        if step % update_interval == 0 and not step == 0:
            # расчитаем прошедшее время
            elapsed = format_time(time.time() - t0)
            
            # расчитаем оставшееся время.
            steps_per_sec = (time.time() - t0) / step
            remaining_sec = steps_per_sec * (len(train_py_inputs) - step)
            remaining = format_time(remaining_sec)

            # вывод прогресса.
            print('  Батч {:>7,}  из  {:>7,}.    Прошло: {:}.  Осталось: {:}'.format(step, len(train_py_inputs), elapsed, remaining))

        # перенсём наши батчи на GPU
        b_input_ids = train_py_inputs[step].to(device)
        b_input_mask = train_py_attn_masks[step].to(device)
        b_labels = train_py_labels[step].to(device)

        # збрасываем градиенты
        model.zero_grad()        

        loss, logits = model(b_input_ids, 
                             #token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # расчёт потери по всем батчам
        total_train_loss += loss.item()

        # Вычисляем градиент
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Обновляем параметры оптимизатора
        optimizer.step()

        # Обновим в оптимизаторе скорость обучения.
        scheduler.step()

    # Рассчитаем среднюю потерю по всем батчам
    avg_train_loss = total_train_loss / len(train_py_inputs)            
    
    # измеряем затраченное время на эпоху
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Тренировка эпохи заняла: {:}".format(training_time))
        
    # Записываем всю статистику тренировки.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Training Time': training_time,
        }
    )

print("")
print("Тренировка окончена!")

print("Тренировка заняла {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Эпоха 1 / 1 ========
Тренировка на 7,979 батчей...
  Батч     800  из    7,979.    Прошло: 0:02:04.  Осталось: 0:18:30
  Батч   1,600  из    7,979.    Прошло: 0:04:04.  Осталось: 0:16:11
  Батч   2,400  из    7,979.    Прошло: 0:06:10.  Осталось: 0:14:19
  Батч   3,200  из    7,979.    Прошло: 0:08:11.  Осталось: 0:12:13
  Батч   4,000  из    7,979.    Прошло: 0:10:08.  Осталось: 0:10:05
  Батч   4,800  из    7,979.    Прошло: 0:12:06.  Осталось: 0:08:01
  Батч   5,600  из    7,979.    Прошло: 0:14:06.  Осталось: 0:05:59
  Батч   6,400  из    7,979.    Прошло: 0:16:02.  Осталось: 0:03:57
  Батч   7,200  из    7,979.    Прошло: 0:18:01.  Осталось: 0:01:57

  Average training loss: 0.10
  Тренировка эпохи заняла: 0:20:02

Тренировка окончена!
Тренировка заняла 0:20:02 (h:mm:ss)


Создадим smart батчи для тестовой выборки

In [36]:
(test_py_inputs, test_py_attn_masks, test_py_labels) = make_smart_batches(test_text, test_labels, batch_size)

Создадим Smart Batches из 31,915 текстов с размером батча 16...

Токенизируем 31,915 текстов...
  Токенизировано 0 текстов.
  Токенизировано 3,000 текстов.
  Токенизировано 6,000 текстов.
  Токенизировано 9,000 текстов.
  Токенизировано 12,000 текстов.
  Токенизировано 15,000 текстов.
  Токенизировано 18,000 текстов.
  Токенизировано 21,000 текстов.
  Токенизировано 24,000 текстов.
  Токенизировано 27,000 текстов.
  Токенизировано 30,000 текстов.
DONE.
    31,915 текстов

    31,915 текстов после сортировки

Выборка батчей размера 16...

  DONE - 1,995 батчей выбрано.

Padding последовательностей в каждом батче...
  DONE.


Сделаем предсказания на тестовых батчах.

In [37]:
print('Предскажем классы для {:,} тестовых сэмплов...'.format(len(test_labels)))

# Переведём модель в другой режим
model.eval()

predictions , true_labels = [], []

update_interval = good_update_interval(total_iters=len(test_py_inputs), num_desired_updates=10)

t0 = time.time()


for step in range(0, len(test_py_inputs)):

    if step % update_interval == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        
        steps_per_sec = (time.time() - t0) / step
        remaining_sec = steps_per_sec * (len(test_py_inputs) - step)
        remaining = format_time(remaining_sec)

        print('  Батч {:>7,}  из  {:>7,}.    Прошло: {:}.  Осталось: {:}'.format(step, len(test_py_inputs), elapsed, remaining))

    b_input_ids = test_py_inputs[step].to(device)
    b_input_mask = test_py_attn_masks[step].to(device)
    b_labels = test_py_labels[step].to(device)
  
    with torch.no_grad():
        # вычисляем предсказания
        outputs = model(b_input_ids, 
                        #token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]

    # переносим предсказания на CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
  
    # сохраняем предсказания
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Предскажем классы для 31,915 тестовых сэмплов...
  Батч     200  из    1,995.    Прошло: 0:00:10.  Осталось: 0:01:33
  Батч     400  из    1,995.    Прошло: 0:00:19.  Осталось: 0:01:15
  Батч     600  из    1,995.    Прошло: 0:00:28.  Осталось: 0:01:04
  Батч     800  из    1,995.    Прошло: 0:00:38.  Осталось: 0:00:57
  Батч   1,000  из    1,995.    Прошло: 0:00:48.  Осталось: 0:00:48
  Батч   1,200  из    1,995.    Прошло: 0:00:56.  Осталось: 0:00:37
  Батч   1,400  из    1,995.    Прошло: 0:01:04.  Осталось: 0:00:27
  Батч   1,600  из    1,995.    Прошло: 0:01:14.  Осталось: 0:00:18
  Батч   1,800  из    1,995.    Прошло: 0:01:24.  Осталось: 0:00:09
    DONE.


In [38]:
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

# Choose the label with the highest score as our prediction.
preds = np.argmax(predictions, axis=1).flatten()

In [39]:
preds[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [40]:
true_labels[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [41]:
print('DistilBertClassification F1-score: {}'.format(f1_score(true_labels, preds)))

DistilBertClassification F1-score: 0.8385332904470891


На DistillBertClassification нам удалось достигнуть F1-score 0.84 на тестовой выборке, это достаточно высокий показатель. И так как нам повезло с GPU в google colab модель натренировалась более чем быстро, если локально без GPU это время занимало чуть ли не 13 часов, то тут модель справилась за 10 минут, что не может не радовать, так же оптимизация батчей дала достаточно сильный прирост при тренировке и предсказании.

**Вывод:** мы получили два разных подхода к предобработке текстов и предсказыванию классов, логистическая регрессия вместе с TF-IDF справились достаточно быстро даже на CPU, хоть и нужна была предварительная обработка данных, а вот Bert, хоть там и не требуется ручная предварительная обработка текстов, так быстро на CPU не запустишь, он будет работать в разы дольше, чем логистическая регрессия, иногда доходило до 13 часов, если не делать смарт батчи, отсортированные по длине последовательности, так что без GPU здесь не обойтись, но Bert оказался сильно точнее, чем логистическая регрессия, примерно на 10 пунктов. Так что при выборе надо учитывать множество факторов, если вы хотите запускать вашу модель на легковесном сервере без GPU, то вам подходит метод с логистической регрессией, если же вы хотите большую точность предсказаний и у вас есть возможность иметь сервер с хорошим GPU, то используейте BERT.